## 手写二维卷积：从训练时间、预测精度、Loss变化等角度分析实验结果（最好使用图表展示）

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
import torch
from torch import nn
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F 
import os

 ## 读取图片文件

In [4]:
def getFileList(dir, FileList,ext=None):
    """
    获取文件夹及子文件夹中的文件列表
    dir: 文件夹跟目录
    ext:扩展名
    返回:文件路径列表
    """
    newDir=dir
    if os.path.isfile(dir):  # os.path.isfile(dir)判断某一对象(需提供绝对路径)是否为文件
        if ext is None:
            FileList.append(dir) # 将文件添加到FileList中
        else:
            if ext in dir[-3:]:
                FileList.append(dir)  # 去掉拓展名之后在添加到FileList中
    elif os.path.isdir(dir): # os.path.isdir()：判断某一对象(需提供绝对路径)是否为目录
        for s in os.listdir(dir):  # os.listdir()：返回一个列表，其中包含有指定路径下的目录和文件的名称
            """
            有时系统中会出现一个desktop.ini的隐藏文件会被读入
            """
            if s[-3:]=="jpg":
                newDir=os.path.join(dir,s)  # 拼接路径
                getFileList(newDir,FileList,ext)
        return FileList

In [5]:
# 检索文件
org_img_folder=r"D:\jupyter-notebook\8-15 实验3\dataset"  # bus car truck

total_imglist=[]
for folder in os.listdir(org_img_folder):
    new_org=os.path.join(org_img_folder,folder)
    imgList = getFileList(new_org,[])
    total_imglist.append(imgList)
    print('本次执行检索到 '+ folder +" "+str(len(imgList))+' 张图像\n')

本次执行检索到 bus 218 张图像

本次执行检索到 car 779 张图像

本次执行检索到 truck 360 张图像



In [6]:
bus_imglist=total_imglist[0]
car_imglist=total_imglist[1]
truck_imglist=total_imglist[2]

def tensor_list(imglist):
    count=0
    for array_img in imglist:
        img=Image.open(array_img)
        img=img.resize((64,64),Image.ANTIALIAS)
        img=np.array(img)
        img=np.expand_dims(img,0) # 从 100*100*3 变成 1*100*100*3
        # 归一化
        img=img/255 
        #1*H*W*C    1*C*H*W   0 1 2 3 ==> 0 3  1 2  X轴用0表示，Y轴用1表示；Z轴用2来表示；
        tensor_img=torch.tensor(np.transpose(img,(0,3,1,2)))
        if count==0:
            imglist=tensor_img
        else:
            imglist=torch.cat((imglist,tensor_img),0)
        count+=1
    return imglist

## 划分数据集

In [7]:
# 训练集
bus_tensor=tensor_list(bus_imglist)
train_bus_tensor=bus_tensor[:int(bus_tensor.shape[0]*0.7)]


car_tensor=tensor_list(car_imglist)
train_car_tensor=car_tensor[:int(car_tensor.shape[0]*0.7)]

truck_tensor=tensor_list(truck_imglist)
train_truck_tensor=truck_tensor[:int(truck_tensor.shape[0]*0.7)]

print(train_bus_tensor.shape) # 152 3 100 100
print(train_car_tensor.shape) # 545 3 100 100
print(train_truck_tensor.shape) # 251

train_total_tensor=torch.cat((train_bus_tensor,train_car_tensor,train_truck_tensor),0)
print(train_total_tensor.shape)

torch.Size([152, 3, 64, 64])
torch.Size([545, 3, 64, 64])
torch.Size([251, 3, 64, 64])
torch.Size([948, 3, 64, 64])


In [8]:
# 测试集
test_bus_tensor=bus_tensor[int(bus_tensor.shape[0]*0.7):]
test_car_tensor=car_tensor[int(car_tensor.shape[0]*0.7):]
test_truck_tensor=truck_tensor[int(truck_tensor.shape[0]*0.7):]
test_total_tensor=torch.cat((test_bus_tensor,test_car_tensor,test_truck_tensor),0)
print(test_total_tensor.shape)

torch.Size([409, 3, 64, 64])


## 给类别分组 0 1 2 

In [9]:
train_bus_labels=torch.zeros(train_bus_tensor.shape[0])  # bus 归为0类
train_car_labels=torch.ones(train_car_tensor.shape[0])   # car 归为1类
train_truck_labels=torch.ones(train_truck_tensor.shape[0])+1   # truck 归为2类

train_total_labels=torch.cat((train_bus_labels,train_car_labels,train_truck_labels),0)
print(train_total_labels.shape)


test_bus_labels=torch.zeros(test_bus_tensor.shape[0])  # bus 归为0类
test_car_labels=torch.ones(test_car_tensor.shape[0])   # car 归为1类
test_truck_labels=torch.ones(test_truck_tensor.shape[0])+1   # truck 归为2类

test_total_labels=torch.cat((test_bus_labels,test_car_labels,test_truck_labels),0)
print(test_total_labels.shape)

torch.Size([948])
torch.Size([409])


## 读取数据集

In [10]:
import torch.utils.data as Data

num_classes=3  # 3类
epochs=3

lr=0.01
batch_size=100
device=torch.device("cuda:0") # 使用device

# 将训练数据的特征和标签组合
train_dataset = Data.TensorDataset(train_total_tensor, train_total_labels)
# 把 dataset 放入 DataLoader
train_iter = Data.DataLoader(
    dataset=train_dataset, # torch TensorDataset format
    batch_size=batch_size, # mini batch size
    shuffle=True, # 是否打乱数据 (训练集一般需要进行打乱)
    num_workers=0, # 多线程来读数据，注意在Windows下需要设置为0
)

# 将训练数据的特征和标签组合
test_dataset = Data.TensorDataset(test_total_tensor, test_total_labels)
# 把 dataset 放入 DataLoader
test_iter = Data.DataLoader(
    dataset=test_dataset, # torch TensorDataset format
    batch_size=batch_size, # mini batch size
    shuffle=True, # 是否打乱数据 (训练集一般需要进行打乱)
    num_workers=0, # 多线程来读数据，注意在Windows下需要设置为0
)

## 自定义卷积层

In [10]:
def corr2d(X,K):    # 这个X就是我读取的测试集，为多通道输入输出
    """
    X：输入：shape(H,W)
    K：卷积核：shape(k_h,k_w)
    """
    batch_size,H,W=X.shape
    k_h,k_w=K.shape
    
    # 初始化结果矩阵
    Y=torch.zeros((batch_size,H - k_h + 1,W - k_w + 1)).to(device)
    
    from tqdm import tqdm  # 添加进度提示信息
    for i in tqdm(range(Y.shape[1])):   # 不知道这个批量加到那个维度了
        for j in range(Y.shape[2]):
            Y[:,i,j]=(X[:,i:i+k_h, j:j+k_w]*K).sum() 
    return Y

def corr2d_multi_in(X,K):
# """
# X: shape(batch_size,C_in,H,W)
# K:shape(C_in,k_h,k_w)
# Y:shape(batch_size,H_out,W_out)
# """
    res=corr2d(X[:,0,:,:],K[0,:,:])
    
    for i in range(1,X.shape[1]):  # 这里应该改成X.shape[3]才是通道数
        # 按通道相加
        res+=corr2d(X[:,i,:,:],K[i,:,:])
    return res

def corr2d_multi_in_out(X,K):
    """
    X:shape(batch_size,C_in,H,W)
    K:shape(C_out,c_in,h,w)
    Y:shape(batch_size,C_out,H_out,W_out)
    """
    return torch.stack([corr2d_multi_in(X,k) for k in K],dim=1)

## 封装卷积层

In [11]:
class MyConv2D(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size):
        super(MyConv2D,self).__init__()
        ## 初始化卷积层参数： K+b
        if isinstance(kernel_size,int):
            kernel_size=(kernel_size,kernel_size)
        self.weight=nn.Parameter(torch.randn((out_channels,in_channels)+ kernel_size))
        self.bias=nn.Parameter(torch.randn(out_channels,1,1))
        
    def forward(self,x):
        """
        x: shape(batch_size,C_in,H,W)
        """
        return corr2d_multi_in_out(x,self.weight)+self.bias

## 二维网络net

In [12]:
"""
输入为 3*64*64
输入通道3 输出通道32  k=3*3

输出大小：H-Kh+1=64-3+1=62
"""
class MyConvModule(nn.Module):
    def __init__(self):
        super(MyConvModule,self).__init__()
        ### 定义一层卷积
        self.conv=nn.Sequential(
            MyConv2D(in_channels=3,out_channels=32,kernel_size=3),
            nn.BatchNorm2d(32),  # 每一个通道归一化
            nn.ReLU(inplace=True)
        )
        ### 输出层
        self.fc=nn.Linear(32,num_classes)
        
    def forward(self,x):
        # 图片先经过一层卷积，输出shape(bactch_size,C_out,H,W)
        out=self.conv(x)
        # 使用平均池化层，每一个图片的大小是62*62
        out=F.avg_pool2d(out,62)
        # 将张量out从shape(batch_size*32*1*1)==>shaoe(batch*32)
        out=out.squeeze()
        # fc
        out=self.fc(out)
        return out

### 初始化+损失函数+优化器

In [13]:
## 模型定义
net=MyConvModule().to(device)
## loss函数
criterion=nn.CrossEntropyLoss()
## 使用Adam优化器
optimizer=torch.optim.Adam(net.parameters(),lr=lr)

## 训练函数

In [14]:
def train_epoch(net,data_loader,device):
    net.train()  ## 当前模式为训练模式
    train_batch_num=len(data_loader)  # 记录有多少个batch
    total_loss=0 # 记录loss
    correct=0    # 记录共有多个个样本被正确分类
    sample_num=0  # 记录样本总数
    
    ### 遍历每一个batch进行训练
    for batch_idx,(data,target)  in enumerate(data_loader):
        # 将图片放入指定的device中
        data=data.to(device).float()
        # 将图片标签放入指定的device中
        target=target.to(device).long()
        #将当前梯度清0
        optimizer.zero_grad()
        # 使用模型计算结果
        output=net(data)
        # 计算损失
        loss=criterion(output,target)
        # 进行反向传播
        loss.backward()
        optimizer.step()
        #累加Loss
        total_loss+=loss.item()
        # 找出每个样本最大的idx，即代表类别
        prediction=torch.argmax(output,1)
        # 统计正确的类别数量
        correct+=(prediction==target).sum().item()
        # 累加当前样本总数
        sample_num+=len(prediction)
    # 计算平均的loss与准确率
    loss=total_loss/train_batch_num
    acc=correct/sample_num
    return loss,acc

## 测试函数

In [15]:
def test_epoch(net,data_loader,device):
    net.eval()  ## 当前模式为训练模式
    test_batch_num=len(data_loader)  # 记录有多少个batch
    total_loss=0 # 记录loss
    correct=0    # 记录共有多个个样本被正确分类
    sample_num=0  # 记录样本总数
    
    
    with torch.no_grad():
        ### 遍历每一个batch进行训练
        for batch_idx,(data,target)  in enumerate(data_loader):
            # 将图片放入指定的device中
            data=data.to(device).float()
            # 将图片标签放入指定的device中
            target=target.to(device).long()
            
            output=net(data)
            # 计算损失
            loss=criterion(output,target)
        
            #累加Loss
            total_loss+=loss.item()
            # 找出每个样本最大的idx，即代表类别
            prediction=torch.argmax(output,1)
            # 统计正确的类别数量
            correct+=(prediction==target).sum().item()
            # 累加当前样本总数
            sample_num+=len(prediction)
    # 计算平均的loss与准确率
    loss=total_loss/test_batch_num
    acc=correct/sample_num
    return loss,acc

## 开始训练

In [ ]:
train_loss_list=[]
train_acc_list=[]
test_loss_list=[]
test_acc_list=[]

# 进行训练
for epoch in range(epochs):
    # 在训练集上训练
    train_loss,train_acc=train_epoch(net,train_iter,device=device)
    # 在测试集上验证
    test_loss,test_acc=test_epoch(net,test_iter,device=device)
    
    ## 保存各个指标
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
   
    test_loss_list.append(test_loss)
    test_acc_list.append(test_acc)
    
    print(f"epoch:{epoch}\t train_loss:{train_loss:.4f}\t"
          f"train_acc:{train_acc} \t"
          f"test_loss:{test_loss:.4f}\t  test_acc:{test_acc}")

100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 81.05it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 79.14it/s]


  0%|                                                                                           | 0/62 [00:00<?, ?it/s]